In [1]:
import os
import re
import sys
import time
import random
import warnings
import collections
from dateutil.relativedelta import relativedelta
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500
pd.options.display.max_colwidth = None

%load_ext autoreload
%autoreload 2

# Script to parse supplemental file response from UHC

In [2]:
file_path = '/Users/bp/workspace/cb/data/00795_UHC_ASMP_DIRECT_20231220105150_RESUB1.txt'

### Columns from the file we built

In [3]:
dx_cols = [f'DX_{i}' for i in range(40)]
columns = [
    'SEG TYPE',
    'REF #',
    'LAST NAME',
    'FIRST NAME',
    'MI',
    'DOB',
    'MEMBER ID- need either member ID or MBI',
    'Retrieval NPI (Internal Use)',
    'GENDER',
    'STATE CODE',
    'MBI-  need either member ID or MBI',
    'FDOS',
    'TDOS',
    'BILL TYPE- Institutional Only',
    'NU Indicator (Internal Use)',
    'PROV ID',
    'NPI',
    'PROV TYPE',
    'FACILITY NM- required for Institutional',
    'PROV LAST NAME- Required for Professional',
    'PROV FIRST NAME- required for Professional',
    'CMS SPECIALTY- Required for Professional',
    'TAX ID',
    'CPT- Professional and Hospital Outpatient only',
    'REV CODE - Required for Institutional',
    'SERVICE FDOS',
    'SERVICE TDOS',
    'POS- Professional only',
    'ICD INDIC',
    'RA Code- Required for Professional',
    'Chart Barcode (Internal Use)',
    'Chart Enc Key (Internal Use)',
    'Chart DX Key  (Internal Use)',
    'Contract ID (Tufts use only)',
    'Mem Street Address ',
    'Mem Address 2',
    'Mem City',
    'Mem State',
    'Mem Zip Code',
    'CLAIMID/PCN',
] + dx_cols

### response

In [4]:
cols = [re.sub('[^0-9a-zA-Z]+', '_', c.lower()) for c in columns+['err_id', 'error']]
df = pd.read_csv(file_path, sep='|', header=None, skipfooter=1, skiprows=1, names=cols, engine='python')
df.head()

,seg_type,ref_,last_name,first_name,mi,dob,member_id_need_either_member_id_or_mbi,retrieval_npi_internal_use_,gender,state_code,mbi_need_either_member_id_or_mbi,fdos,tdos,bill_type_institutional_only,nu_indicator_internal_use_,prov_id,npi,prov_type,facility_nm_required_for_institutional,prov_last_name_required_for_professional,prov_first_name_required_for_professional,cms_specialty_required_for_professional,tax_id,cpt_professional_and_hospital_outpatient_only,rev_code_required_for_institutional,service_fdos,service_tdos,pos_professional_only,icd_indic,ra_code_required_for_professional,chart_barcode_internal_use_,chart_enc_key_internal_use_,chart_dx_key_internal_use_,contract_id_tufts_use_only_,mem_street_address_,mem_address_2,mem_city,mem_state,mem_zip_code,claimid_pcn,dx_0,dx_1,dx_2,dx_3,dx_4,dx_5,dx_6,dx_7,dx_8,dx_9,dx_10,dx_11,dx_12,dx_13,dx_14,dx_15,dx_16,dx_17,dx_18,dx_19,dx_20,dx_21,dx_22,dx_23,dx_24,dx_25,dx_26,dx_27,dx_28,dx_29,dx_30,dx_31,dx_32,dx_33,dx_34,dx_35,dx_36,dx_37,dx_38,dx_39,err_id,error
0,DTL,619209105080476,Allen,Robert,NaN,04/17/1962,NaN,NaN,M,NaN,7RD3Y38RG96,06/14/2023,06/14/2023,NaN,NaN,NaN,1629335120,NaN,NaN,Balmer,Leanne,50,84-2590508,99214,NaN,NaN,NaN,10,0,A,NaN,NaN,NaN,NaN,506 Green Street,NaN,LEBANON,TN,37087,NaN,K219,G800,J302,E785,I10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513475-423810482,err18-Blank MemberID;err40-Member Not Found in Plan Member;
1,DTL,675350386442396,Leggins,Eddie,NaN,04/21/1940,NaN,NaN,M,NaN,7C64H85KM83,10/11/2023,10/11/2023,NaN,NaN,NaN,1376196717,NaN,NaN,Hammond,Piper,50,84-2590508,99212,NaN,NaN,NaN,10,0,A,NaN,NaN,NaN,NaN,2076 Rile St,NaN,Memphis,TN,38109,NaN,Z7984,R600,E1151,G8194,I872,I509,E261,Z794,Z7409,E785,E1169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513475-423777319,err18-Blank MemberID;err40-Member Not Found in Plan Member;
2,DTL,647304561164444,Blevins,Barbara,NaN,12/06/1953,NaN,NaN,F,NaN,7EE2PM1GH80,08/15/2023,08/15/2023,NaN,NaN,NaN,1629335120,NaN,NaN,Balmer,Leanne,50,84-2590508,99212,NaN,NaN,NaN,10,0,A,NaN,NaN,NaN,NaN,3064 HIGHWAY 11 E LOT 35,NaN,BLUFF CITY,TN,37618,NaN,NaN,IMO0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513475-423777109,err27-Invalid Diag Cd DOS(DX 2);
3,DTL,683395266707612,Vaughn,Randy,NaN,06/03/1955,NaN,NaN,M,NaN,3XQ4A30ER93,10/26/2023,10/26/2023,NaN,NaN,NaN,1316428527,NaN,NaN,Murray,Candice,50,84-2590508,99204,NaN,NaN,NaN,10,0,A,NaN,NaN,NaN,NaN,233 Vaal Ave,NaN,Memphis,TN,38109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IMO0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513475-423850907,err27-Invalid Diag Cd DOS(DX 15);
4,DTL,651235198632092,Soule,Jeanette,NaN,02/24/1939,NaN,NaN,F,NaN,5RG7YT9AY73,08/23/2023,08/23/2023,NaN,NaN,NaN,1376196717,NaN,NaN,Hammond,Piper,50,84-2590508,99214,NaN,NaN,NaN,10,0,A,NaN,NaN,NaN,NaN,928 Highway 100 west,NaN,Centerville,TN,37033,NaN,I719,J45909,M797,Z6822,F0150,F32A,N390,I69351,M069,F321,E785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513475-423971265,err42-Member Data Mismatch (LN);


In [5]:
df.error.value_counts()

err18-Blank MemberID;err40-Member Not Found in Plan Member;                                     32
err42-Member Data Mismatch (LN);                                                                21
err27-Invalid Diag Cd DOS(DX 1);                                                                15
err27-Invalid Diag Cd DOS(DX 8);                                                                10
err27-Invalid Diag Cd DOS(DX 4);                                                                 8
err27-Invalid Diag Cd DOS(DX 5);                                                                 8
err27-Invalid Diag Cd DOS(DX 9);                                                                 7
err27-Invalid Diag Cd DOS(DX 2);                                                                 7
err18-Blank MemberID;err35-Member Not Found in Eligibility;                                      6
err27-Invalid Diag Cd DOS(DX 10);                                                                6
err27-Inva

In [15]:
df.loc[df.error == 'err42-Member Data Mismatch (LN);', ['mbi_need_either_member_id_or_mbi']]

,mbi_need_either_member_id_or_mbi
4,5RG7YT9AY73
8,1UC7JJ6TM59
17,7K65HV5GV71
18,9AU9A34YK12
22,8Y18D31PR53
24,3T82UG8JV12
29,5CN8Y14YU34
32,1MT0C67XC34
43,4UJ1HR2CC27
44,1A98V71CV24


In [11]:
# HDR|8.9|00795|00795_UHC_ASMP_DIRECT_20220328162146_RESUB1.txt|UHC|DIRECT|P|Y

In [7]:
conn = cb_utils.get_engine(source="analytics")
df.to_sql("uhc_supp_file_errors_20230627", conn, schema='junk', index=False, method='multi', chunksize=1000)

577

In [14]:
df.loc[df.error == 'err24_ Invalid or Not Risk Adjustable CPT; err45-Blank or Invalid POS;'].cpt_professional_and_hospital_outpatient_only.value_counts()

98966    117
99211     20
99422      1
99421      1
Name: cpt_professional_and_hospital_outpatient_only, dtype: int64

In [8]:
df.loc[df.error.str.contains('Diag Cd'), ['error', 'fdos'] + df.columns[-42:].tolist()].head(20)
# 'M545' # not valid for billing
# 'I714' # not valid for billing
# 'D7582' # not valid for billing
# 'F03911' # too new `23
# 'Z7985' # too new `23

,error,fdos,dx_0,dx_1,dx_2,dx_3,dx_4,dx_5,dx_6,dx_7,dx_8,dx_9,dx_10,dx_11,dx_12,dx_13,dx_14,dx_15,dx_16,dx_17,dx_18,dx_19,dx_20,dx_21,dx_22,dx_23,dx_24,dx_25,dx_26,dx_27,dx_28,dx_29,dx_30,dx_31,dx_32,dx_33,dx_34,dx_35,dx_36,dx_37,dx_38,dx_39,err_id,error
3,err27-Invalid Diag Cd DOS (DX 05);,03/25/2022,NaN,NaN,NaN,NaN,M545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484525-10000342,err27-Invalid Diag Cd DOS (DX 05);
6,err27-Invalid Diag Cd DOS (DX 04);,06/29/2022,NaN,NaN,NaN,M545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484525-10000657,err27-Invalid Diag Cd DOS (DX 04);
93,err27-Invalid Diag Cd DOS (DX 04);,08/24/2022,NaN,NaN,NaN,F03911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484525-10003631,err27-Invalid Diag Cd DOS (DX 04);
144,err27-Invalid Diag Cd DOS (DX 15);,09/16/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Z7985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484525-10004340,err27-Invalid Diag Cd DOS (DX 15);
160,err22-Blank MBI; err27-Invalid Diag Cd DOS (DX 04); err40-Member not found in plan membership;,12/02/2022,K219,B182,F0150,I714,I209,G40909,I7102,I509,E785,I110,I69351,I252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484525-10005250,err22-Blank MBI; err27-Invalid Diag Cd DOS (DX 04); err40-Member not found in plan membership;
161,err27-Invalid Diag Cd DOS (DX 03);,01/13/2023,NaN,NaN,D7582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484525-10005266,err27-Invalid Diag Cd DOS (DX 03);
170,err27-Invalid Diag Cd DOS (DX 07);,09/19/2022,NaN,NaN,NaN,NaN,NaN,NaN,Z7985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484525-10006063,err27-Invalid Diag Cd DOS (DX 07);
188,err27-Invalid Diag Cd DOS (DX 06);,06/14/2023,NaN,NaN,NaN,NaN,NaN,Z9115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484525-10006815,err27-Invalid Diag Cd DOS (DX 06);


In [33]:
df.loc[df.error.str.contains('MemberID')]

,seg_type,ref_,last_name,first_name,mi,dob,member_id_need_either_member_id_or_mbi,retrieval_npi_internal_use_,gender,state_code,mbi_need_either_member_id_or_mbi,fdos,tdos,bill_type_institutional_only,nu_indicator_internal_use_,prov_id,npi,prov_type,facility_nm_required_for_institutional,prov_last_name_required_for_professional,prov_first_name_required_for_professional,cms_specialty_required_for_professional,tax_id,cpt_professional_and_hospital_outpatient_only,rev_code_required_for_institutional,service_fdos,service_tdos,pos_professional_only,icd_indic,ra_code_required_for_professional,chart_barcode_internal_use_,chart_enc_key_internal_use_,chart_dx_key_internal_use_,contract_id_tufts_use_only_,mem_street_address_,mem_address_2,mem_city,mem_state,mem_zip_code,claimid_pcn,dx_0,dx_1,dx_2,dx_3,dx_4,dx_5,dx_6,dx_7,dx_8,dx_9,dx_10,dx_11,dx_12,dx_13,dx_14,dx_15,dx_16,dx_17,dx_18,dx_19,dx_20,dx_21,dx_22,dx_23,dx_24,dx_25,dx_26,dx_27,dx_28,dx_29,dx_30,dx_31,dx_32,dx_33,dx_34,dx_35,dx_36,dx_37,dx_38,dx_39,err_id,error
139,DTL,432289519042716,Raymer,Daisy,NaN,03/28/1929,NaN,NaN,F,NaN,6JY3DN2XN80,03/31/2022,03/31/2022,NaN,NaN,NaN,1811345341,NaN,NaN,Freeman,Megan,50.0,84-2590508,99213,NaN,NaN,NaN,11.0,0,A,NaN,NaN,NaN,NaN,1685 Bunker Hill Rd,NaN,Cookeville,TN,38506.0,NaN,R627,F0390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483473-10000140,err18-Blank MemberID; err40-Member not found in plan membership; err45-Blank or Invalid POS;
484,DTL,502836801765532,Friday,Airfearer,NaN,09/04/1954,NaN,NaN,F,NaN,1EC3N32FD72,08/19/2022,08/19/2022,NaN,NaN,NaN,1740410703,NaN,NaN,Bolden,Lacey,50.0,84-2590508,99214,NaN,NaN,NaN,10.0,0,A,NaN,NaN,NaN,NaN,108 N Auburndale St Apt 409,NaN,Memphis,TN,38104.0,NaN,R569,S062X5S,F0390,I69951,N3281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483473-10000485,err18-Blank MemberID; err40-Member not found in plan membership; err45-Blank or Invalid POS;
485,DTL,512259942252700,Friday,Airfearer,NaN,09/04/1954,NaN,NaN,F,NaN,1EC3N32FD72,09/15/2022,09/15/2022,NaN,NaN,NaN,1740410703,NaN,NaN,Bolden,Lacey,50.0,84-2590508,99213,NaN,NaN,NaN,10.0,0,A,NaN,NaN,NaN,NaN,108 N Auburndale St Apt 409,NaN,Memphis,TN,38104.0,NaN,L089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483473-10000486,err18-Blank MemberID; err40-Member not found in plan membership; err45-Blank or Invalid POS;
486,DTL,581423447670940,Friday,Airfearer,NaN,09/04/1954,NaN,NaN,F,NaN,1EC3N32FD72,03/22/2023,03/22/2023,NaN,NaN,NaN,1811345341,NaN,NaN,Freeman,Megan,50.0,84-2590508,99214,NaN,NaN,NaN,10.0,0,A,NaN,NaN,NaN,NaN,108 N Auburndale St Apt 409,NaN,Memphis,TN,38104.0,NaN,R569,S062X5S,F0390,I69951,N3281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483473-10000487,err18-Blank MemberID; err40-Member not found in plan membership; err45-Blank or Invalid POS;
517,DTL,614794085728412,Mason,Alberta,NaN,08/17/1927,NaN,NaN,F,NaN,1CA2Q38GP29,06/06/2023,06/06/2023,NaN,NaN,NaN,1841796091,NaN,NaN,Holland,Jamie,50.0,84-2590508,99203,NaN,NaN,NaN,10.0,0,A,NaN,NaN,NaN,NaN,5185 Highway 57 Apt 24d,NaN,Rossville,TN,38066.0,NaN,K219,I152,J449,Z6824,E1159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483473-10000518,err18-Blank MemberID; err40-Member not found in plan membership; err45-Blank or Invalid POS;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

13995